In [1]:
import bempp.api
from bempp.api.assembly.blocked_operator import BlockedOperator
import math
import numpy as np
import scipy 
import cmath
from numba import objmode
from numpy.linalg import slogdet
from bempp.api.operators.far_field import helmholtz as helmholtz_farfield
import matplotlib.pyplot as plt
import bempp.api.shapes.fractals

In [2]:
h = 0.05
dist = 3.0

sphere1 = bempp.api.shapes.sphere(h=h, origin=((dist/2) + 1, 0, 0))
sphere2 = bempp.api.shapes.sphere(h=h, origin=(-((dist/2) + 1), 0, 0))

space_sphere1 = bempp.api.function_space(sphere1, "P", 1)
space_sphere2 = bempp.api.function_space(sphere2, "P", 1)

for N_wn in [60, 70]:

    for ubound_wn in [4, 5, 10]:
        lbound_wn = 0
        quad_points = np.linspace(np.exp(-ubound_wn), np.exp(-lbound_wn), N_wn)
        results = []
        for index2, point2 in enumerate(quad_points):

            wavenumber = -1j * np.log(point2)

            slp11 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere1, space_sphere1, wavenumber)
            slp12 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere1, space_sphere1, wavenumber)
            slp21 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere2 , space_sphere2 , wavenumber)
            slp22 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere2 , space_sphere2 , wavenumber)

            mat11 = slp11.weak_form().A
            mat12 = slp12.weak_form().A
            mat21 = slp21.weak_form().A
            mat22 = slp22.weak_form().A
            mat12_zero = np.zeros((mat11.shape[0],mat12.shape[1]))
            mat21_zero = np.zeros((mat22.shape[0],mat11.shape[1]))

            mat = np.block([[mat11,mat12],[mat21,mat22]])  
            mat11_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat11), np.eye(mat11.shape[0]))
            mat22_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat22), np.eye(mat22.shape[0]))

            inv_free_mat2 = np.block([[mat11_inv,  mat12_zero],
                                  [mat21_zero, mat22_inv]])  

            combined_inverse_free = inv_free_mat2.dot(mat)
            sign, logdet = slogdet(combined_inverse_free)

            results.append(logdet)
        print(N_wn, np.trapz(-np.array(results) / quad_points, quad_points) / (2 * np.pi))

60 0.0010197178089833834
60 0.0010197427270832254
60 0.0010197572510269783
70 0.0010194354417361838
70 0.0010194536623577494
70 0.0010194642809266236


In [ ]:
Z = 3.0
ub = 4, 5, 10
h = 0.1
20 0.0010262850145902295
20 0.0010265241885411326
20 0.0010266636526601919
30 0.0010205257946712752
30 0.0010206286618993689
30 0.0010206886327574242
40 0.0010185868194453056
40 0.0010186437350640477
40 0.0010186769091438343
50 0.001017707480681256
50 0.0010177435467482296
50 0.001017764563604227
60 0.0010172358288712952
60 0.0010172607092608147
60 0.0010172752045446121
70 0.0010169538887274232
70 0.0010169720817567861
70 0.0010169826785344145


CasE_30_01 = [0.0010262850145902295, 0.0010265241885411326, 0.0010266636526601919, 0.0010205257946712752, 0.0010206286618993689, 
              0.0010206886327574242, 0.0010185868194453056, 0.0010186437350640477, 0.0010186769091438343, 0.001017707480681256,
             0.0010177435467482296, 0.001017764563604227, 0.0010172358288712952, 0.0010172607092608147, 0.0010172752045446121,
             0.0010169538887274232, 0.0010169720817567861, 0.0010169826785344145]
CasE_30_005 = [0.0010287807049933532, 0.0010290202414384875, 0.001029159937793544, 0.0010230127588125688, 0.0010231157819142155,
               0.0010231758573272338, 0.001021070846156894, 0.0010211278480090952, 0.0010211610825080011, 0.0010201901752515626,
               0.001020226295980908, 0.0010202473528090172, 0.0010197178089833834, 0.0010197427270832254, 0.0010197572510269783, 
               0.0010194354417361838, 0.0010194536623577494, 0.0010194642809266236]
CasE_30_extrap = [0.0010296126017943943, 0.0010298522590709391, 0.0010299920328379947, 0.0010238417468596667, 0.0010239448219191644, 
                  0.001024004932183837, 0.0010218988550607566, 0.0010219558856574443, 0.0010219891402960567, 0.0010210177401083315, 
                  0.0010210538790584674, 0.0010210749492106138, 0.0010205451356874128, 0.0010205700663573623, 0.0010205845998544336, 
                  0.0010202626260724375, 0.001020280855891404, 0.0010202914817240268]

h = 0.05

20 0.0010287807049933532
20 0.0010290202414384875
20 0.001029159937793544
30 0.0010230127588125688
30 0.0010231157819142155
30 0.0010231758573272338
40 0.001021070846156894
40 0.0010211278480090952
40 0.0010211610825080011
50 0.0010201901752515626
50 0.001020226295980908
50 0.0010202473528090172
60 0.0010197178089833834
60 0.0010197427270832254
60 0.0010197572510269783
70 0.0010194354417361838
70 0.0010194536623577494
70 0.0010194642809266236
#===============================================================

Z = 1.5
ub = 8, 9, 10
h = 0.1
20 0.0046686599804245865
20 0.004668665914574311
20 0.004668668075048978
30 0.0046617736090401605
30 0.00466177614825127
30 0.0046617770675181636
40 0.004659457120827237
40 0.0046594585205674945
40 0.0046594590244243205
50 0.0046584069094583805
50 0.0046584077935745176
50 0.004658408110001049
60 0.0046578436931204795
60 0.0046578443012380115
60 0.0046578445176278455
70 0.0046575070475643
70 0.004657507491000173
70 0.004657507647834673


CasE_15_01 = [0.0046686599804245865, 0.004668665914574311, 0.004668668075048978, 0.0046617736090401605, 0.00466177614825127, 
              0.0046617770675181636, 0.004659457120827237, 0.0046594585205674945, 0.0046594590244243205, 0.0046584069094583805, 
              0.0046584077935745176, 0.004658408110001049, 0.0046578436931204795, 0.0046578443012380115, 0.0046578445176278455,
              0.0046575070475643, 0.004657507491000173, 0.004657507647834673]
CasE_15_005 = [0.004684502449871548, 0.00468450843388044, 0.004684510596741076, 0.004677607732183618, 0.004677610299632613, 
               0.004677611218939785, 0.004675288427113692, 0.004675289846273479, 0.004675290349533151, 0.0046742369364865905,
               0.004674237835265853, 0.004674238150950219, 0.004673673033427651, 0.004673673653229866, 0.004673673868845562,
               0.004673335977107332, 0.004673336430187327, 0.004673336586305]
CasE_15_extrap = [0.004689783273020535, 0.004689789273649149, 0.004689791437305108, 0.004682885773231437, 0.00468288835009306, 
                  0.004682889269413658, 0.004680565529209178, 0.004680566954842141, 0.004680567457902761, 0.004679513612162661, 
                  0.004679514515829632, 0.0046795148312666094, 0.004678949480196708, 0.0046789501038938175, 0.004678950319251468, 
                  0.00467861228695501, 0.004678612743249711, 0.004678612899128443]
h = 0.05

20 0.004684502449871548
20 0.00468450843388044
20 0.004684510596741076
30 0.004677607732183618
30 0.004677610299632613
30 0.004677611218939785
40 0.004675288427113692
40 0.004675289846273479
40 0.004675290349533151
50 0.0046742369364865905
50 0.004674237835265853
50 0.004674238150950219
60 0.004673673033427651
60 0.004673673653229866
60 0.004673673868845562
70 0.004673335977107332
70 0.004673336430187327
70 0.004673336586305
#========================================================================

Z = 0.5
ub = 25
h = 0.1

20 0.043786492319013035
30 0.043809876777666575
40 0.04382059915645993
50 0.04382662875780533
60 0.04383044357362744
70 0.043833050800340626

CasE_05_01 = [0.043786492319013035, 0.043809876777666575, 0.04382059915645993, 0.04382662875780533, 0.04383044357362744, 0.043833050800340626]
CasE_05_005 = [0.044305867203324606, 0.044267365994235475, 0.04424810540638025, 0.044236460971181286, 0.04422862972751019, 0.044222988057969315]
CasE_05_extrap = [0.044478992164761796, 0.04441986239975844, 0.044390607489687026, 0.04437307170897327, 0.04436135844547111, 0.04435296714384555]

h = 0.05

20 0.044305867203324606
30 0.044267365994235475
40 0.04424810540638025
50 0.044236460971181286
60 0.04422862972751019
70 0.044222988057969315

In [14]:
1/(12*50**2)

3.3333333333333335e-05

In [11]:
h = 0.1
dist = 0.5

sphere1 = bempp.api.shapes.sphere(h=h, origin=((dist/2) + 1, 0, 0))
sphere2 = bempp.api.shapes.sphere(h=h, origin=(-((dist/2) + 1), 0, 0))

space_sphere1 = bempp.api.function_space(sphere1, "P", 1)
space_sphere2 = bempp.api.function_space(sphere2, "P", 1)       

wavenumber = 1j * 25

slp11 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere1, space_sphere1, wavenumber)
slp12 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere1, space_sphere1, wavenumber)
slp21 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere2 , space_sphere2 , wavenumber)
slp22 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere2 , space_sphere2 , wavenumber)

mat11 = slp11.weak_form().A
mat12 = slp12.weak_form().A
mat21 = slp21.weak_form().A
mat22 = slp22.weak_form().A
mat12_zero = np.zeros((mat11.shape[0],mat12.shape[1]))
mat21_zero = np.zeros((mat22.shape[0],mat11.shape[1]))

mat = np.block([[mat11,mat12],[mat21,mat22]])  
mat11_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat11), np.eye(mat11.shape[0]))
mat22_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat22), np.eye(mat22.shape[0]))

inv_free_mat2 = np.block([[mat11_inv,  mat12_zero],
                      [mat21_zero, mat22_inv]])  

combined_inverse_free = inv_free_mat2.dot(mat)
sign, logdet = slogdet(combined_inverse_free)

print(logdet)


-2.829181333652453e-12


In [9]:
np.exp(-2*dist*5) 

9.357622968840175e-14

In [12]:
(-2.712274849159258e-13 + (-2.829181333652453e-12))*5/2

-7.751022046420947e-12

In [ ]:
CasE_05 = [0.044142920889365764, 0.04378745428754526, 0.043307825800813826, 0.0425274735585427] 
CasE_15 = [0.004684510596741076, 0.004668668075048978, 0.0046463126764854845, 0.004608327156406329] 
CasE_30 = [0.001029159937793544, 0.0010266636526601919, 0.0010231501929812861, 0.0010171903056843653]

In [ ]:
Rel_err_05 = [abs(CasE_05[0] - CasE_05[1])/CasE_05[0], abs(CasE_05[0] - CasE_05[2])/CasE_05[0], abs(CasE_05[0] - CasE_05[3])/CasE_05[0]]
Rel_err_15 = [abs(CasE_15[0] - CasE_15[1])/CasE_15[0], abs(CasE_15[0] - CasE_15[2])/CasE_15[0], abs(CasE_15[0] - CasE_15[3])/CasE_15[0]]
Rel_err_30 = [abs(CasE_30[0] - CasE_30[1])/CasE_30[0], abs(CasE_30[0] - CasE_30[2])/CasE_30[0], abs(CasE_30[0] - CasE_30[3])/CasE_30[0]]

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(21, 6))
ax1.plot([0.1, 0.15, 0.2], Rel_err_05,'*-')
ax1.set_xlabel('Grid size $h$', fontsize = 20)
ax1.set_ylabel('Relative error', fontsize = 15)
ax1.set_title('dist = 0.5', fontsize = 20)

ax2.plot([0.1, 0.15, 0.2], Rel_err_15,'*-')
ax2.set_xlabel('Grid size $h$', fontsize = 20)
ax2.set_ylabel('Relative error', fontsize = 15)
ax2.set_title('dist = 1.5', fontsize = 20)

ax3.plot([0.1, 0.15, 0.2], Rel_err_15,'*-')
ax3.set_xlabel('Grid size $h$', fontsize = 20)
ax3.set_ylabel('Relative error', fontsize = 15)
ax3.set_title('dist = 3.0', fontsize = 20)


fig.tight_layout()
fig.suptitle('Relative error in the scalar case', fontsize = 25)
fig.subplots_adjust(top=0.85)
fig.savefig('CasE_scalar.pdf')